In [ ]:
import pandas as pd
import torch
import gc
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 0. CONFIGURATION DES MODÈLES
# ==========================================
models_config = [
    # --- LE DUEL FRANÇAIS ---
    {"name": "Solon (Spécialiste Juridique)", "id": "OrdalieTech/Solon-embeddings-large-0.1"},
    {"name": "Marsilia (Généraliste FR)", "id": "sujet-ai/Marsilia-Embeddings-FR-Large"},

    # --- LES CHALLENGERS INTERNATIONAUX ---
    {"name": "Snowflake Arctic v2", "id": "Snowflake/snowflake-arctic-embed-m-v2.0"},
    {"name": "BGE-M3 (Multilingue)", "id": "BAAI/bge-m3"},
    {"name": "GTE-Multilingual", "id": "Alibaba-NLP/gte-multilingual-base"},
    {"name": "E5-Large Instruct", "id": "intfloat/multilingual-e5-large-instruct"}
]

# ==========================================
# 1. DATASET PAIRES POSITIVES (RETRIEVAL)
# ==========================================
# Objectif : Trouver la bonne définition parmi 85 distracteurs (Simulation Moteur de Recherche).

# A. Le Corpus (Termes + Définitions)
corpus_termes = [
    "TVA", "TVA Intracommunautaire", "IR (Impôt Revenu)", "Prélèvement à la source",
    "Taxe d'habitation", "Taxe Foncière", "PFU (Flat Tax)", "CSG", "Droits de succession",
    "Niche fiscale", "Quotient familial", "IS (Impôt Sociétés)", "CFE", "CVAE", "TASCOM",
    "Amortissement comptable", "Provision", "Bilan comptable", "Compte de résultat", "BFR",
    "Kbis", "SIREN", "SIRET", "Assiette fiscale", "Liquidation", "Recouvrement",
    "Rescrit fiscal", "Dégrèvement", "Redressement fiscal", "ATD (Avis Tiers Détenteur)",
    "Prescription fiscale", "Contrôle sur pièces", "Loi de Finances (PLF)", "Déficit budgétaire",
    "Dette publique", "BOP", "AE (Autorisations d'Engagement)", "CP (Crédits de Paiement)",
    "Engagement juridique", "Service fait", "Mandatement", "Ordonnateur", "Comptable public",
    "Fongibilité", "Titre de recette", "Chorus Pro", "Marché public", "Appel d'offres",
    "CCTP", "CCAG", "Recours gracieux", "Cadastre", "Journal Officiel", "Décret",
    "Jurisprudence", "PIB", "Inflation", "Plus-value", "OAT", "Taux directeur"
]

corpus_definitions = [
    "Impôt indirect sur la consommation payé par le consommateur final.",
    "Numéro d'identification pour les échanges au sein de l'UE.",
    "Impôt direct progressif sur l'ensemble des revenus du foyer.",
    "Mode de recouvrement de l'impôt au moment du versement du revenu.",
    "Impôt local dû par l'occupant d'un logement au 1er janvier.",
    "Impôt local dû par les propriétaires de biens immobiliers.",
    "Taux fixe de 30% appliqué aux revenus du capital.",
    "Prélèvement finançant la sécurité sociale (maladie, retraite).",
    "Impôt sur la transmission du patrimoine d'un défunt.",
    "Avantage fiscal dérogatoire diminuant l'impôt dû.",
    "Système de parts selon la composition du foyer fiscal.",
    "Impôt sur le bénéfice annuel des entreprises en France.",
    "Impôt local des entreprises basé sur la valeur locative foncière.",
    "Cotisation sur la Valeur Ajoutée des entreprises (si CA élevé).",
    "Taxe sur les surfaces commerciales de détail.",
    "Constatation de la perte de valeur d'un actif (usure/temps).",
    "Passif incertain (montant/échéance) couvrant un risque probable.",
    "Synthèse du patrimoine (Actif/Passif) à un instant T.",
    "Présentation des produits et charges (Résultat) de l'exercice.",
    "Décalage de trésorerie né du cycle d'exploitation.",
    "Extrait officiel prouvant l'existence juridique d'une entreprise.",
    "Identifiant unique de l'entreprise (9 chiffres).",
    "Identifiant de l'établissement (14 chiffres).",
    "Base de calcul d'un impôt.",
    "Calcul exact de l'impôt exigible une fois l'assiette fixée.",
    "Procédures pour obtenir le paiement dû au Trésor.",
    "Réponse de l'administration engageant sa parole sur une situation.",
    "Suppression partielle de l'impôt accordée par l'administration.",
    "Rectification d'une imposition insuffisante ou omise.",
    "Saisie d'une somme due directement chez un tiers (banque).",
    "Délai au-delà duquel l'administration ne peut plus réclamer.",
    "Vérification des déclarations depuis les bureaux du fisc.",
    "Loi prévoyant et autorisant le budget de l'État.",
    "Solde négatif du budget (Dépenses > Recettes).",
    "Cumul des engagements financiers de l'État.",
    "Regroupement de crédits pour un responsable de programme.",
    "Limite supérieure des dépenses pouvant être engagées juridiquement.",
    "Limite supérieure des dépenses pouvant être payées dans l'année.",
    "Acte créant une obligation (bon de commande).",
    "Certification que la prestation a été réalisée.",
    "Ordre de payer donné au comptable.",
    "Agent qui décide de la dépense mais ne manie pas les fonds.",
    "Agent seul habilité à manier les fonds publics.",
    "Utilisation des crédits d'un titre pour un autre.",
    "Acte émis pour recouvrer une créance publique.",
    "Portail de facturation électronique public.",
    "Contrat onéreux pour répondre aux besoins d'un acheteur public.",
    "Procédure de choix de l'offre sans négociation.",
    "Cahier des Clauses Techniques Particulières.",
    "Cahier des Clauses Administratives Générales.",
    "Demande de réexamen à l'auteur de la décision.",
    "Registre définissant les propriétés foncières.",
    "Publication officielle des lois et décrets.",
    "Acte réglementaire pris par le Premier ministre.",
    "Ensemble des décisions de justice servant de référence.",
    "Richesse produite sur le territoire national.",
    "Hausse générale et durable des prix.",
    "Gain réalisé lors de la vente d'un bien.",
    "Titre d'emprunt d'État à long terme.",
    "Taux de la Banque Centrale influençant le crédit."
]

# B. Les Questions "Usager" (Query) qui doivent matcher le Corpus (Target)
dataset_queries = [
    ("Je paie quoi sur mon salaire tous les mois ?", "IR (Impôt Revenu)"),
    ("C'est quoi la taxe sur les achats en magasin ?", "TVA"),
    ("L'État me prend de l'argent sur l'héritage de mon père.", "Droits de succession"),
    ("Je suis proprio, je reçois un avis en automne.", "Taxe Foncière"),
    ("C'est quoi le papier qui prouve que ma boite existe ?", "Kbis"),
    ("Ma société a fait du profit, je dois payer quoi ?", "IS (Impôt Sociétés)"),
    ("Je n'ai pas assez de cash pour payer mes fournisseurs.", "BFR"),
    ("Le fisc a fait une erreur, je demande un remboursement.", "Dégrèvement"),
    ("On a saisi mon compte bancaire pour impayé.", "ATD (Avis Tiers Détenteur)"),
    ("C'est quoi la dette totale de la France ?", "Dette publique"),
    ("Qui a le droit de toucher l'argent public ?", "Comptable public"),
    ("Je cherche les spécifications techniques du marché.", "CCTP"),
    ("L'État dépense trop par rapport à ce qu'il gagne.", "Déficit budgétaire"),
    ("Les prix augmentent partout.", "Inflation"),
    ("Où sont publiées les nouvelles lois ?", "Journal Officiel")
]

# ==========================================
# 2. TRIPLETS (SEPARATION / LOGIQUE)
# ==========================================
# Objectif : Forcer la séparation des concepts ambigus.
dataset_triplets = [
    # Acronymes Fiscalité & Compta
    ("IS (Impôt Sociétés)", "Bénéfice", "Chiffre d'affaires"),
    ("TVA", "Consommation", "Revenu"),
    ("SIREN", "Entreprise", "Établissement"),
    ("AE (Autorisation Engagement)", "Juridique", "Trésorerie"),
    ("CP (Crédit Paiement)", "Trésorerie", "Juridique"),
    ("Bilan", "Patrimoine", "Rentabilité"),
    ("Compte de Résultat", "Flux", "Stock"),

    # Rôles et Procédures
    ("Ordonnateur", "Décideur", "Payeur"),
    ("Comptable Public", "Payeur", "Décideur"),
    ("Mandatement", "Ordre de payer", "Paiement effectif"),
    ("Service fait", "Réalité", "Commande"),

    # Concepts juridiques proches
    ("Décret", "Gouvernement", "Parlement"),
    ("Loi", "Parlement", "Gouvernement"),
    ("Jurisprudence", "Juge", "Législateur"),
    ("Recours gracieux", "Administration", "Juge"),

    # Économie / Dette
    ("Déficit", "Flux annuel", "Dette cumulée"),
    ("Dette", "Stock", "Flux"),
    ("OAT", "Emprunt", "Impôt"),
    ("Amortissement", "Perte valeur", "Sortie cash")
]

# ==========================================
# 3. MOTEUR D'ÉVALUATION
# ==========================================

def evaluate_model(model, name):
    # Préfixes pour les modèles d'instruction (E5)
    p_q = "query: " if "e5" in name.lower() or "instruct" in name.lower() else ""
    p_d = "passage: " if "e5" in name.lower() or "instruct" in name.lower() else ""

    print(f"   🔹 Encodage en cours...")

    # --- PARTIE 1 : PAIRES POSITIVES (RETRIEVAL) ---
    # Encodage du corpus (Définitions)
    corpus_txt = [p_d + d for d in corpus_definitions]
    emb_corpus = model.encode(corpus_txt, convert_to_tensor=True, show_progress_bar=False)

    # Encodage des Queries Type 1 (Termes)
    termes_txt = [p_q + t for t in corpus_termes]
    emb_termes = model.encode(termes_txt, convert_to_tensor=True, show_progress_bar=False)

    # Encodage des Queries Type 2 (Questions Usager)
    qa_queries = [p_q + q[0] for q in dataset_queries]
    qa_targets_str = [q[1] for q in dataset_queries]
    emb_qa = model.encode(qa_queries, convert_to_tensor=True, show_progress_bar=False)

    # Calcul Score 1 (Dictionnaire : Terme -> Déf)
    scores_td = util.cos_sim(emb_termes, emb_corpus)
    acc_td = 0
    for i in range(len(corpus_termes)):
        if torch.argmax(scores_td[i]).item() == i:
            acc_td += 1

    # Calcul Score 2 (FAQ : Question -> Déf)
    scores_qd = util.cos_sim(emb_qa, emb_corpus)
    acc_qa = 0
    for i, target_term in enumerate(qa_targets_str):
        target_idx = corpus_termes.index(target_term)
        if torch.argmax(scores_qd[i]).item() == target_idx:
            acc_qa += 1

    score_paires = ((acc_td / len(corpus_termes)) + (acc_qa / len(dataset_queries))) / 2 * 100

    # --- PARTIE 2 : TRIPLETS (SEPARATION) ---
    anchors = [p_q + t[0] for t in dataset_triplets]
    positives = [p_d + t[1] for t in dataset_triplets]
    negatives = [p_d + t[2] for t in dataset_triplets]

    emb_a = model.encode(anchors, convert_to_tensor=True, show_progress_bar=False)
    emb_p = model.encode(positives, convert_to_tensor=True, show_progress_bar=False)
    emb_n = model.encode(negatives, convert_to_tensor=True, show_progress_bar=False)

    sim_pos = torch.nn.functional.cosine_similarity(emb_a, emb_p)
    sim_neg = torch.nn.functional.cosine_similarity(emb_a, emb_n)

    acc_triplets = 0
    for i in range(len(dataset_triplets)):
        if sim_pos[i] > sim_neg[i]:
            acc_triplets += 1

    score_triplets = (acc_triplets / len(dataset_triplets)) * 100

    return {
        "Modèle": name,
        "1. Paires Positives (Retrieval)": score_paires,
        "2. Triplets (Séparation)": score_triplets,
        "Moyenne": (score_paires + score_triplets) / 2
    }

# ==========================================
# 4. EXÉCUTION
# ==========================================
results = []
print(f"🚀 LANCEMENT DU BENCHMARK (MARSILIA vs LE MONDE)")

for cfg in models_config:
    print(f"\n📥 Chargement : {cfg['name']}")
    try:
        # Note : SentenceTransformer gère automatiquement le chargement de sujet-ai/Marsilia...
        model = SentenceTransformer(cfg['id'], trust_remote_code=True)
        res = evaluate_model(model, cfg['name'])
        results.append(res)
        print(f"   ✅ Moyenne : {res['Moyenne']:.1f}%")

        # Nettoyage mémoire
        del model
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"   ❌ Erreur avec {cfg['name']}: {e}")

# AFFICHAGE DU TABLEAU
df = pd.DataFrame(results).sort_values(by="Moyenne", ascending=False)
print("\n🏆 RÉSULTATS FINAUX 🏆")
print(df.to_markdown(index=False, floatfmt=".1f"))

🚀 LANCEMENT DU BENCHMARK (MARSILIA vs LE MONDE)

📥 Chargement : Solon (Spécialiste Juridique)
   🔹 Encodage en cours...
   ✅ Moyenne : 46.7%

📥 Chargement : Marsilia (Généraliste FR)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

   🔹 Encodage en cours...
   ✅ Moyenne : 51.0%

📥 Chargement : Snowflake Arctic v2
   ❌ Erreur avec Snowflake Arctic v2: please install xformers

📥 Chargement : BGE-M3 (Multilingue)
   🔹 Encodage en cours...
   ✅ Moyenne : 36.9%

📥 Chargement : GTE-Multilingual


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


   🔹 Encodage en cours...
   ✅ Moyenne : 43.9%

📥 Chargement : E5-Large Instruct
   🔹 Encodage en cours...
   ✅ Moyenne : 40.9%

🏆 RÉSULTATS FINAUX 🏆
| Modèle                        |   1. Paires Positives (Retrieval) |   2. Triplets (Séparation) |   Moyenne |
|:------------------------------|----------------------------------:|---------------------------:|----------:|
| Marsilia (Généraliste FR)     |                              44.2 |                       57.9 |      51.0 |
| Solon (Spécialiste Juridique) |                              40.8 |                       52.6 |      46.7 |
| GTE-Multilingual              |                              30.0 |                       57.9 |      43.9 |
| E5-Large Instruct             |                              45.0 |                       36.8 |      40.9 |
| BGE-M3 (Multilingue)          |                              31.7 |                       42.1 |      36.9 |


In [ ]:
pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 5.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import gc
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 0. CONFIGURATION DES MODÈLES
# ==========================================
models_config = [
    # --- LE DUEL FRANÇAIS ---
    {"name": "Solon (Spécialiste Juridique)", "id": "OrdalieTech/Solon-embeddings-large-0.1"},
    {"name": "Marsilia (Généraliste FR)", "id": "sujet-ai/Marsilia-Embeddings-FR-Large"},

    # --- LES CHALLENGERS INTERNATIONAUX ---
    {"name": "Snowflake Arctic v2", "id": "Snowflake/snowflake-arctic-embed-m-v2.0"},
    {"name": "BGE-M3 (Multilingue)", "id": "BAAI/bge-m3"},
    {"name": "GTE-Multilingual", "id": "Alibaba-NLP/gte-multilingual-base"},
    {"name": "E5-Large Instruct", "id": "intfloat/multilingual-e5-large-instruct"}
]

# ==========================================
# 1. GIGA DATASET BERCY (135+ Termes)
# ==========================================

# A. CORPUS (Termes)
corpus_termes = [
    # --- FISCALITÉ PARTICULIERS (DGFIP) ---
    "IR (Impôt sur le Revenu)", "TVA", "Taxe d'habitation", "Taxe Foncière",
    "Prélèvement à la source", "Quotient familial", "PFU (Flat Tax)", "CSG", "CRDS",
    "Droits de succession", "IFI (Impôt Fortune Immobilière)", "Niche fiscale",
    "Déclaration 2042", "Avis d'imposition", "Revenu Fiscal de Référence (RFR)",
    "Parts fiscales", "Décote", "Prime d'activité", "PAS (Prélèvement A la Source)",

    # --- FISCALITÉ ENTREPRISES & PRO ---
    "IS (Impôt sur les Sociétés)", "TVA Intracommunautaire", "CFE", "CVAE", "CET",
    "TASCOM", "TGAP", "Liasses fiscales", "Déclaration 2035", "Crédit d'Impôt Recherche (CIR)",
    "JEI (Jeune Entreprise Innovante)", "DAS2", "Imprimé Fiscal Unique (IFU)",
    "Amortissement réputé différé", "Carry-back (Report en arrière)", "Régime mère-fille",
    "Intégration fiscale", "Prix de transfert", "Taxes sur les salaires", "Organic",

    # --- CONTRÔLE & PROCÉDURES FISCALES ---
    "Contrôle sur pièces", "Vérification de comptabilité", "ESFP", "Rescrit fiscal",
    "Redressement fiscal", "Assiette fiscale", "Liquidation", "Recouvrement",
    "Dégrèvement", "Contentieux fiscal", "Sursis de paiement", "Prescription fiscale",
    "Abus de droit", "Acte anormal de gestion", "Fraude fiscale", "Fraude Carrousel",
    "Droit de communication", "Avis à Tiers Détenteur (ATD)", "Opposition administrative",
    "Gage du Trésor",

    # --- DOUANES & COMMERCE INTERNATIONAL (DGDDI) ---
    "Dédouanement", "Octroi de mer", "Accises", "Droits de douane", "TARIC",
    "Nomenclature douanière", "OEA (Opérateur Économique Agréé)", "DELTA",
    "Contrefaçon", "Franchise douanière", "Admission temporaire", "Entrepôt sous douane",
    "Déclaration en douane (DAU)", "Origine préférentielle", "Régime de transit",
    "Dette douanière", "Contrôle douanier", "Biens à double usage", "Incivilités",

    # --- GESTION PUBLIQUE & GBCP (Comptabilité de l'État) ---
    "Loi de Finances (PLF)", "Loi de Règlement", "BOP", "UO (Unité Opérationnelle)",
    "AE (Autorisations d'Engagement)", "CP (Crédits de Paiement)", "Engagement Juridique (EJ)",
    "Service Fait (SF)", "Demande de Paiement (DP)", "Mandatement", "Ordonnateur",
    "Comptable public", "Régisseur d'avances", "Titularisation", "Fongibilité asymétrique",
    "Dépense publique", "Recette non fiscale", "Titre de perception", "RAP (Reste à Payer)",
    "RAR (Reste à Recouvrer)", "Compte Général de l'État (CGE)", "Chorus Coeur",

    # --- MARCHÉS PUBLICS (Commande Publique) ---
    "Marché public", "Accord-cadre", "Appel d'offres", "MAPA", "Sourcing",
    "DUME", "DC1 / DC2", "CCTP", "CCAP", "BPU (Bordereau Prix Unitaires)",
    "DQE (Détail Quantitatif Estimatif)", "Avenant", "Sous-traitance", "Co-traitance",
    "Référé précontractuel", "BOAMP", "JOUE", "Seuils européens", "Mieux-disant",

    # --- RESSOURCES HUMAINES (Fonction Publique) ---
    "Catégorie A/B/C", "Corps", "Grade", "Échelon", "Titulaire", "Contractuel",
    "Détachement", "Mise à disposition", "Disponibilité", "Mutation",
    "RIFSEEP", "IFSE", "CIA (Complément Indemnitaire)", "GIPA", "Point d'indice",
    "Grille indiciaire", "Avancement de grade", "Concours interne/externe",
    "CET (Compte Épargne Temps)", "Positions administratives",

    # --- MACROÉCONOMIE & FINANCES ---
    "PIB", "Inflation", "Dette publique", "Déficit public", "Déficit structurel",
    "OAT (Obligation Assimilable du Trésor)", "Spread", "Agence France Trésor (AFT)",
    "Pacte de stabilité", "Zone Euro", "Taux directeur", "BCE", "Balance commerciale",
    "Solde primaire", "Prélèvements obligatoires"
]

# B. CORPUS (Définitions correspondantes)
corpus_definitions = [
    "Impôt direct progressif calculé au niveau du foyer fiscal sur l'ensemble des revenus.",
    "Impôt indirect sur la consommation payé par le consommateur final.",
    "Impôt local dû par l'occupant d'un logement au 1er janvier (résidence principale/secondaire).",
    "Impôt local dû par les propriétaires de biens immobiliers bâtis ou non bâtis.",
    "Mode de recouvrement de l'impôt consistant à le faire payer au moment du versement du revenu.",
    "Système divisant le revenu imposable en nombre de parts selon la composition du foyer.",
    "Prélèvement Forfaitaire Unique appliquant un taux fixe (30%) aux revenus du capital.",
    "Contribution Sociale Généralisée finançant la protection sociale.",
    "Contribution pour le Remboursement de la Dette Sociale.",
    "Impôt prélevé sur la transmission du patrimoine d'une personne décédée.",
    "Impôt sur la fortune immobilière remplaçant l'ISF, taxant le patrimoine immobilier net.",
    "Avantage fiscal dérogatoire (crédit, réduction) permettant de diminuer l'impôt dû.",
    "Formulaire principal de déclaration des revenus des personnes physiques.",
    "Document officiel indiquant le montant de l'impôt à payer ou le remboursement.",
    "Montant net des revenus retenu pour le calcul de certaines aides et exonérations.",
    "Unités représentatives de la composition du foyer pour le calcul du quotient familial.",
    "Réduction d'impôt accordée aux foyers modestes.",
    "Complément de revenu pour les travailleurs à revenus modestes.",
    "Acronyme pour Prélèvement à la Source.",

    "Impôt prélevé sur le bénéfice annuel des entreprises exploitées en France.",
    "Numéro d'identification fiscal individuel pour les échanges au sein de l'UE.",
    "Cotisation Foncière des Entreprises, basée sur la valeur locative des biens.",
    "Cotisation sur la Valeur Ajoutée des Entreprises, due si CA > 500k€.",
    "Contribution Économique Territoriale (somme de la CFE et de la CVAE).",
    "Taxe sur les surfaces commerciales due par les grands magasins.",
    "Taxe Générale sur les Activités Polluantes.",
    "Ensemble des déclarations fiscales (bilan, résultat) remises par les entreprises.",
    "Déclaration des bénéfices non commerciaux (BNC) pour les professions libérales.",
    "Dispositif fiscal de soutien aux activités de R&D des entreprises.",
    "Statut offrant des exonérations fiscales aux PME investissant en R&D.",
    "Déclaration des honoraires et commissions versés aux tiers.",
    "Document récapitulatif des opérations sur valeurs mobilières (dividendes, intérêts).",
    "Report d'un déficit sur les bénéfices antérieurs pour obtenir une créance.",
    "Report d'un déficit sur un bénéfice antérieur (créance d'impôt).",
    "Régime exonérant les dividendes reçus par une société mère de sa filiale.",
    "Régime permettant de globaliser l'impôt au niveau du groupe (mère et filiales).",
    "Prix pratiqués pour les transactions entre sociétés d'un même groupe (international).",
    "Taxe due par les employeurs non soumis à la TVA sur les rémunérations versées.",
    "Loi organique relative aux lois de finances (Constitution financière).",

    "Vérification de la cohérence des déclarations effectuée depuis les bureaux.",
    "Contrôle fiscal effectué dans les locaux de l'entreprise.",
    "Examen de la Situation Fiscale Personnelle d'un particulier.",
    "Réponse formelle de l'administration engageant sa parole sur une situation.",
    "Opération de rectification d'une imposition insuffisante ou omise.",
    "Base de calcul d'un impôt.",
    "Calcul exact de l'impôt exigible une fois l'assiette fixée.",
    "Ensemble des procédures pour obtenir le paiement d'une somme due.",
    "Suppression ou atténuation de l'impôt accordée par l'administration.",
    "Litige entre le contribuable et l'administration devant le juge.",
    "Délai de paiement accordé au contribuable en cas de réclamation.",
    "Délai au-delà duquel l'administration ne peut plus réclamer l'impôt.",
    "Montage juridique fictif ayant pour seul but d'éluder l'impôt.",
    "Dépense de l'entreprise étrangère à son intérêt.",
    "Soustraction illégale au paiement de l'impôt.",
    "Fraude à la TVA impliquant des sociétés éphémères dans l'UE.",
    "Droit du fisc d'obtenir des infos auprès de tiers (banques, employeurs).",
    "Procédure de saisie d'une somme due par un tiers au débiteur.",
    "Blocage des fonds par l'administration (ex: sur un compte bancaire).",
    "Garantie prise par l'État sur les biens d'un débiteur.",

    "Procédure administrative pour l'entrée ou la sortie de marchandises du territoire.",
    "Taxe spécifique sur les produits importés dans les départements d'outre-mer.",
    "Impôts indirects sur la consommation de certains produits (alcool, tabac, pétrole).",
    "Taxes perçues sur les marchandises importées lors du passage de la frontière.",
    "Tarif intégré des Communautés européennes (codification des marchandises).",
    "Système de classement des marchandises pour déterminer les droits de douane.",
    "Label de confiance douanier accordé aux entreprises fiables.",
    "Système informatique de dédouanement en ligne.",
    "Imitation frauduleuse d'un produit violant un droit de propriété intellectuelle.",
    "Exonération des droits et taxes pour certaines marchandises.",
    "Régime permettant d'importer temporairement sans droits de douane.",
    "Lieu agréé pour stocker des marchandises non dédouanées.",
    "Document Administratif Unique utilisé pour les déclarations douanières.",
    "Statut permettant des droits de douane réduits selon le pays d'origine.",
    "Circulation de marchandises sous surveillance douanière sans paiement immédiat.",
    "Montant des droits de douane et taxes dus à l'importation.",
    "Vérification physique ou documentaire des marchandises par les douaniers.",
    "Produits civils susceptibles d'avoir une utilisation militaire.",
    "Comportements agressifs ou irrespectueux envers les agents.",

    "Acte législatif prévoyant et autorisant le budget de l'État pour l'année à venir.",
    "Loi constatant les résultats financiers de l'année civile écoulée (compte rendu).",
    "Budget Opérationnel de Programme (niveau régional de gestion).",
    "Unité de gestion de base (niveau local) pour l'exécution du budget.",
    "Limite supérieure des dépenses pouvant être engagées juridiquement.",
    "Limite supérieure des dépenses pouvant être payées (décaissées).",
    "Acte par lequel une personne publique crée une obligation (bon de commande).",
    "Certification que la prestation a été réalisée, conditionnant le paiement.",
    "Ordre donné par le service gestionnaire de payer une dépense.",
    "Ordre donné par l'ordonnateur au comptable de payer.",
    "Agent public qui décide de la dépense mais ne manie pas les fonds.",
    "Agent public seul habilité à manier les fonds et payer les dépenses.",
    "Agent chargé de gérer une caisse d'avance pour les menues dépenses.",
    "Intégration définitive d'un fonctionnaire dans un corps.",
    "Possibilité de redéployer des crédits (sauf vers dépenses de personnel).",
    "Sortie d'argent du budget de l'État.",
    "Recette de l'État autre que les impôts (amendes, dividendes).",
    "Document justificatif permettant d'encaisser une recette.",
    "Montant engagé mais non encore payé.",
    "Sommes dues à l'État non encore encaissées.",
    "Bilan comptable de l'État (Actif/Passif) en droits constatés.",
    "Système d'information financière de l'État (ERP SAP).",

    "Contrat onéreux conclu par un acheteur public pour répondre à ses besoins.",
    "Contrat fixant les termes régissant les marchés à passer sur une période.",
    "Procédure formelle de mise en concurrence.",
    "Marché à Procédure Adaptée (montant inférieur aux seuils européens).",
    "Consultation préalable des entreprises pour définir le besoin.",
    "Document Unique de Marché Européen (déclaration sur l'honneur).",
    "Documents de consultation (Candidature / Offre).",
    "Cahier des Clauses Techniques Particulières (spécifications).",
    "Cahier des Clauses Administratives Particulières (juridique).",
    "Liste des prix unitaires applicables au marché.",
    "Simulation de commande pour comparer les prix des candidats.",
    "Modification du contrat de marché public en cours d'exécution.",
    "Exécution d'une partie du marché par un tiers sous la responsabilité du titulaire.",
    "Groupement d'entreprises répondant ensemble à un marché.",
    "Recours juridique d'urgence contre la procédure de passation.",
    "Bulletin Officiel des Annonces des Marchés Publics.",
    "Journal Officiel de l'Union Européenne.",
    "Montants déterminant la procédure de marché applicable.",
    "Critère d'attribution privilégiant le rapport qualité/prix.",

    "Niveau hiérarchique supérieur (encadrement, conception).",
    "Ensemble de fonctionnaires soumis au même statut particulier.",
    "Titre conférant rang et prérogatives dans le corps.",
    "Niveau d'avancement à l'intérieur d'un grade (ancienneté).",
    "Fonctionnaire ayant été titularisé dans un grade.",
    "Agent public non titulaire (CDD ou CDI).",
    "Fonctionnaire exerçant hors de son corps d'origine mais payé par l'accueil.",
    "Fonctionnaire travaillant hors de son administration mais payé par elle.",
    "Cessation temporaire d'activité (sans traitement).",
    "Changement d'affectation ou de résidence administrative.",
    "Régime Indemnitaire tenant compte des Fonctions, Sujétions, Expertise...",
    "Indemnité de Fonctions, de Sujétions et d'Expertise (partie fixe RIFSEEP).",
    "Partie variable du RIFSEEP liée à l'engagement pro.",
    "Garantie Individuelle du Pouvoir d'Achat.",
    "Valeur de base servant au calcul du traitement brut.",
    "Tableau définissant la rémunération selon le grade et l'échelon.",
    "Passage au grade supérieur (au choix ou examen).",
    "Modes de recrutement des fonctionnaires.",
    "Dispositif permettant de stocker des jours de congés.",
    "Situations statutaires du fonctionnaire (activité, détachement, etc.).",

    "Indicateur mesurant la richesse produite sur le territoire.",
    "Hausse générale et durable des prix.",
    "Ensemble des engagements financiers de l'État.",
    "Solde négatif du budget annuel (Flux).",
    "Déficit indépendant de la conjoncture économique.",
    "Titre d'emprunt d'État à moyen ou long terme.",
    "Écart de taux d'intérêt entre deux obligations d'États (ex: France/Allemagne).",
    "Service chargé de gérer la dette et la trésorerie de l'État.",
    "Règles budgétaires européennes (limite déficit/dette).",
    "Pays de l'UE ayant adopté l'euro.",
    "Taux d'intérêt fixé par la Banque Centrale.",
    "Banque Centrale Européenne.",
    "Différence entre les exportations et les importations.",
    "Solde budgétaire avant paiement des intérêts de la dette.",
    "Ensemble des impôts et cotisations sociales versés aux administrations."
]

# C. QUERIES (QUESTIONS UTILISATEUR)
dataset_queries = [
    # USAGERS PARTICULIERS
    ("Je déclare mes revenus sur quel formulaire ?", "Déclaration 2042"),
    ("C'est quoi l'impôt sur la fortune immobilière ?", "IFI (Impôt Fortune Immobilière)"),
    ("Je veux savoir combien je vais payer d'impôt cette année.", "Avis d'imposition"),
    ("Mon employeur prélève directement l'impôt sur ma paie, c'est quoi ?", "PAS (Prélèvement A la Source)"),
    ("J'ai fait un don à une asso, j'ai droit à quoi ?", "Niche fiscale"), # ou Réduction

    # ENTREPRISES
    ("Ma boite investit dans la recherche, on a une aide ?", "Crédit d'Impôt Recherche (CIR)"),
    ("Je dois déclarer les honoraires que je verse à mes avocats.", "DAS2"),
    ("Je cherche le code pour classer mes marchandises à l'import.", "TARIC"),
    ("Mon entreprise veut le label de confiance douanier.", "OEA (Opérateur Économique Agréé)"),
    ("C'est quoi la taxe sur les bureaux ?", "CFE"),

    # AGENTS PUBLICS (RH)
    ("Je change de ministère mais je garde mon statut, je suis en...", "Détachement"),
    ("C'est quoi ma prime liée à ma manière de servir ?", "CIA (Complément Indemnitaire)"),
    ("Je veux poser des jours sur mon compte épargne temps.", "CET (Compte Épargne Temps)"),
    ("Quel est le montant de base de mon salaire ?", "Point d'indice"),

    # GESTIONNAIRES PUBLICS (FINANCES)
    ("Je dois réserver les crédits pour un marché de 3 ans.", "AE (Autorisations d'Engagement)"),
    ("Je dois payer la facture de 2024.", "CP (Crédits de Paiement)"),
    ("Je valide que la prestation a été livrée conforme.", "Service Fait (SF)"),
    ("Je déplace des crédits du titre 2 vers le titre 3.", "Fongibilité asymétrique"),

    # MARCHÉS PUBLICS
    ("On lance un marché de 40 000 euros, quelle procédure ?", "MAPA"),
    ("Je veux répondre à un marché public européen.", "DUME"),
    ("L'entreprise la moins chère n'est pas la meilleure techniquement.", "Mieux-disant"),
    ("Je modifie le contrat du marché en cours de route.", "Avenant")
]

# ==========================================
# 2. TRIPLETS (SEPARATION / LOGIQUE)
# ==========================================
dataset_triplets = [
    # Distinctions Techniques
    ("AE (Autorisation Engagement)", "Juridique/Réservation", "Trésorerie/Décaissement"),
    ("CP (Crédit Paiement)", "Décaissement", "Engagement"),
    ("Ordonnateur", "Décideur", "Comptable"),
    ("Comptable Public", "Payeur", "Ordonnateur"),
    ("Service Fait", "Livraison conforme", "Commande"),

    # Douanes vs Fiscalité
    ("TVA", "Intérieur/National", "Frontière/Douane"),
    ("Droits de douane", "Frontière/Import", "Intérieur/TVA"),
    ("Octroi de mer", "Outre-mer", "Métropole"),

    # RH Fonction Publique
    ("Détachement", "Carrière continue", "Disponibilité"), # Dispo = carrière arrêtée
    ("Mise à disposition", "Reste dans son corps", "Détachement"),
    ("Titulaire", "CDI Public", "Contractuel"),
    ("RIFSEEP", "Prime", "Traitement indiciaire"),

    # Marchés Publics
    ("MAPA", "Procédure allégée", "Appel d'offres formalisé"),
    ("Mieux-disant", "Qualité/Prix", "Moins-disant"), # Moins-disant = prix seul
    ("CCTP", "Technique", "Administratif"),
    ("CCAP", "Administratif/Juridique", "Technique"),

    # Acronymes Pièges
    ("CIR", "Recherche", "Compétitivité"), # CIR vs CICE
    ("DAS2", "Honoraires", "TVA"),
    ("DUME", "Candidature", "Facturation"),
    ("BOAMP", "Publicité", "Exécution")
]

# ==========================================
# 3. MOTEUR D'ÉVALUATION
# ==========================================

def evaluate_model(model, name):
    # Préfixes pour les modèles d'instruction (E5)
    p_q = "query: " if "e5" in name.lower() or "instruct" in name.lower() else ""
    p_d = "passage: " if "e5" in name.lower() or "instruct" in name.lower() else ""

    print(f"   🔹 Encodage en cours...")

    # --- PARTIE 1 : PAIRES POSITIVES (RETRIEVAL) ---
    # Encodage du corpus (Définitions)
    corpus_txt = [p_d + d for d in corpus_definitions]
    emb_corpus = model.encode(corpus_txt, convert_to_tensor=True, show_progress_bar=False)

    # Encodage des Queries Type 1 (Termes)
    termes_txt = [p_q + t for t in corpus_termes]
    emb_termes = model.encode(termes_txt, convert_to_tensor=True, show_progress_bar=False)

    # Encodage des Queries Type 2 (Questions Usager)
    qa_queries = [p_q + q[0] for q in dataset_queries]
    qa_targets_str = [q[1] for q in dataset_queries]
    emb_qa = model.encode(qa_queries, convert_to_tensor=True, show_progress_bar=False)

    # Calcul Score 1 (Dictionnaire : Terme -> Déf)
    scores_td = util.cos_sim(emb_termes, emb_corpus)
    acc_td = 0
    for i in range(len(corpus_termes)):
        if torch.argmax(scores_td[i]).item() == i:
            acc_td += 1

    # Calcul Score 2 (FAQ : Question -> Déf)
    scores_qd = util.cos_sim(emb_qa, emb_corpus)
    acc_qa = 0
    for i, target_term in enumerate(qa_targets_str):
        try:
            target_idx = corpus_termes.index(target_term)
            if torch.argmax(scores_qd[i]).item() == target_idx:
                acc_qa += 1
        except ValueError:
            # Sécurité au cas où une cible Q&A n'est pas dans le corpus
            continue

    score_paires = ((acc_td / len(corpus_termes)) + (acc_qa / len(dataset_queries))) / 2 * 100

    # --- PARTIE 2 : TRIPLETS (SEPARATION) ---
    anchors = [p_q + t[0] for t in dataset_triplets]
    positives = [p_d + t[1] for t in dataset_triplets]
    negatives = [p_d + t[2] for t in dataset_triplets]

    emb_a = model.encode(anchors, convert_to_tensor=True, show_progress_bar=False)
    emb_p = model.encode(positives, convert_to_tensor=True, show_progress_bar=False)
    emb_n = model.encode(negatives, convert_to_tensor=True, show_progress_bar=False)

    sim_pos = torch.nn.functional.cosine_similarity(emb_a, emb_p)
    sim_neg = torch.nn.functional.cosine_similarity(emb_a, emb_n)

    acc_triplets = 0
    for i in range(len(dataset_triplets)):
        if sim_pos[i] > sim_neg[i]:
            acc_triplets += 1

    score_triplets = (acc_triplets / len(dataset_triplets)) * 100

    return {
        "Modèle": name,
        "1. Paires Positives (Retrieval)": score_paires,
        "2. Triplets (Séparation)": score_triplets,
        "Moyenne": (score_paires + score_triplets) / 2
    }

# ==========================================
# 4. EXÉCUTION
# ==========================================
results = []
print(f"🚀 LANCEMENT DU BENCHMARK (MARSILIA vs LE MONDE - GIGA DATASET)")

for cfg in models_config:
    print(f"\n📥 Chargement : {cfg['name']}")
    try:
        model = SentenceTransformer(cfg['id'], trust_remote_code=True)
        res = evaluate_model(model, cfg['name'])
        results.append(res)
        print(f"   ✅ Moyenne : {res['Moyenne']:.1f}%")

        # Nettoyage mémoire
        del model
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"   ❌ Erreur avec {cfg['name']}: {e}")

# AFFICHAGE DU TABLEAU
df = pd.DataFrame(results).sort_values(by="Moyenne", ascending=False)
print("\n🏆 RÉSULTATS FINAUX 🏆")
print(df.to_markdown(index=False, floatfmt=".1f"))

🚀 LANCEMENT DU BENCHMARK (MARSILIA vs LE MONDE - GIGA DATASET)

📥 Chargement : Solon (Spécialiste Juridique)
   🔹 Encodage en cours...
   ✅ Moyenne : 58.8%

📥 Chargement : Marsilia (Généraliste FR)
   🔹 Encodage en cours...
   ✅ Moyenne : 49.0%

📥 Chargement : Snowflake Arctic v2
   ❌ Erreur avec Snowflake Arctic v2: please install xformers

📥 Chargement : BGE-M3 (Multilingue)
   🔹 Encodage en cours...
   ✅ Moyenne : 30.7%

📥 Chargement : GTE-Multilingual


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


   🔹 Encodage en cours...
   ✅ Moyenne : 51.4%

📥 Chargement : E5-Large Instruct
   🔹 Encodage en cours...
   ✅ Moyenne : 55.2%

🏆 RÉSULTATS FINAUX 🏆
| Modèle                        |   1. Paires Positives (Retrieval) |   2. Triplets (Séparation) |   Moyenne |
|:------------------------------|----------------------------------:|---------------------------:|----------:|
| Solon (Spécialiste Juridique) |                              37.7 |                       80.0 |      58.8 |
| E5-Large Instruct             |                              45.5 |                       65.0 |      55.2 |
| GTE-Multilingual              |                              32.8 |                       70.0 |      51.4 |
| Marsilia (Généraliste FR)     |                              38.0 |                       60.0 |      49.0 |
| BGE-M3 (Multilingue)          |                              31.5 |                       30.0 |      30.7 |
